# Домашнее задание 7: Кластеризация

Анализ методов неконтролируемого обучения на различных датасетах.

In [ ]:
# Импорт необходимых библиотек
import pandas as pd
import numpy as np
import json
import os
from pathlib import Path

# Для визуализации
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn.decomposition import PCA

# Для предобработки
from sklearn.preprocessing import StandardScaler, SimpleImputer
from sklearn.impute import SimpleImputer

# Для кластеризации
from sklearn.cluster import KMeans, DBSCAN, AgglomerativeClustering

# Для оценки качества
from sklearn.metrics import silhouette_score, davies_bouldin_score, calinski_harabasz_score
from sklearn.metrics import adjusted_rand_score

# Параметры визуализации
sns.set_style('whitegrid')
plt.rcParams['figure.figsize'] = (10, 6)

# Создание папок для артефактов
os.makedirs('artifacts/figures', exist_ok=True)
os.makedirs('artifacts/labels', exist_ok=True)

print('Все необходимые библиотеки импортированы')

## Датасет 1: Анализ и кластеризация

### Загрузка и EDA

In [ ]:
# Загрузка датасета 1
df1 = pd.read_csv('data/S07-hw-dataset-01.csv')

print('=== Датасет 1: Информация ===' )
print(f'Размер: {df1.shape}')
print(f'\nПервые строки:')
print(df1.head())
print(f'\nТип данных:')
print(df1.info())
print(f'\nСтатистика:')
print(df1.describe())
print(f'\nПропущенные значения:')
print(df1.isnull().sum())

### Предобработка датасета 1

In [ ]:
# Отделяем sample_id от признаков
sample_ids_1 = df1['sample_id'].copy()
X1_raw = df1.drop('sample_id', axis=1)

# Обработка пропущенных значений
imputer = SimpleImputer(strategy='mean')
X1_imputed = imputer.fit_transform(X1_raw)

# Стандартизация признаков
scaler = StandardScaler()
X1 = scaler.fit_transform(X1_imputed)

print('Датасет 1 предобработан')
print(f'Форма после предобработки: {X1.shape}')
print(f'Среднее: {X1.mean(axis=0)}')
print(f'Стандартное отклонение: {X1.std(axis=0)}')

### KMeans кластеризация для датасета 1

In [ ]:
# Тестирование KMeans с разными k
k_range = range(2, 21)
silhouette_scores_1_km = []
davies_bouldin_scores_1_km = []
calinski_harabasz_scores_1_km = []
kmeans_models_1 = {}

for k in k_range:
    # KMeans кластеризация
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_km = kmeans.fit_predict(X1)
    kmeans_models_1[k] = (kmeans, labels_km)
    
    # Расчет метрик
    sil_score = silhouette_score(X1, labels_km)
    db_score = davies_bouldin_score(X1, labels_km)
    ch_score = calinski_harabasz_score(X1, labels_km)
    
    silhouette_scores_1_km.append(sil_score)
    davies_bouldin_scores_1_km.append(db_score)
    calinski_harabasz_scores_1_km.append(ch_score)
    
    print(f'k={k}: Silhouette={sil_score:.4f}, Davies-Bouldin={db_score:.4f}, Calinski-Harabasz={ch_score:.4f}')

# Выбор лучшего k по silhouette
best_k_1_km = list(k_range)[np.argmax(silhouette_scores_1_km)]
print(f'\nЛучшее k для KMeans: {best_k_1_km}')
print(f'Лучший silhouette score: {max(silhouette_scores_1_km):.4f}')

### DBSCAN кластеризация для датасета 1

In [ ]:
# Тестирование DBSCAN с разными eps
eps_range = np.linspace(0.3, 2.0, 15)
min_samples = 5

silhouette_scores_1_db = []
davies_bouldin_scores_1_db = []
calinski_harabasz_scores_1_db = []
noise_percentages_1_db = []
dbscan_models_1 = {}

for eps in eps_range:
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels_db = dbscan.fit_predict(X1)
    dbscan_models_1[eps] = (dbscan, labels_db)
    
    n_clusters = len(set(labels_db)) - (1 if -1 in labels_db else 0)
    noise_points = np.sum(labels_db == -1)
    noise_percent = (noise_points / len(labels_db)) * 100
    noise_percentages_1_db.append(noise_percent)
    
    if n_clusters > 1 and len(set(labels_db)) > 1:
        sil_score = silhouette_score(X1, labels_db)
        db_score = davies_bouldin_score(X1, labels_db)
        ch_score = calinski_harabasz_score(X1, labels_db)
    else:
        sil_score = -1
        db_score = float('inf')
        ch_score = 0
    
    silhouette_scores_1_db.append(sil_score)
    davies_bouldin_scores_1_db.append(db_score)
    calinski_harabasz_scores_1_db.append(ch_score)
    
    print(f'eps={eps:.2f}: Clusters={n_clusters}, Noise={noise_percent:.2f}%, Silhouette={sil_score:.4f}')

# Выбор лучшего eps по silhouette
best_eps_1_db = eps_range[np.argmax(silhouette_scores_1_db)]
print(f'\nЛучшее eps для DBSCAN: {best_eps_1_db:.2f}')
print(f'Лучший silhouette score: {max(silhouette_scores_1_db):.4f}')

### Визуализация датасета 1

In [ ]:
# PCA для визуализации
pca = PCA(n_components=2)
X1_pca = pca.fit_transform(X1)

# Лучший KMeans результат
best_kmeans_1 = kmeans_models_1[best_k_1_km][0]
labels_best_1_km = kmeans_models_1[best_k_1_km][1]

# PCA scatter plot для KMeans
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(X1_pca[:, 0], X1_pca[:, 1], c=labels_best_1_km, cmap='viridis', s=50, alpha=0.6)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
ax.set_title(f'Датасет 1: KMeans (k={best_k_1_km})')
plt.colorbar(scatter, ax=ax, label='Cluster')
plt.tight_layout()
plt.savefig('artifacts/figures/pca_dataset1_kmeans.png', dpi=100, bbox_inches='tight')
plt.show()
print('PCA plot KMeans сохранен')

In [ ]:
# Метрики vs K для KMeans
fig, axes = plt.subplots(1, 3, figsize=(15, 4))

axes[0].plot(list(k_range), silhouette_scores_1_km, marker='o')
axes[0].set_xlabel('Number of clusters (k)')
axes[0].set_ylabel('Silhouette Score')
axes[0].set_title('KMeans: Silhouette vs k')
axes[0].grid(True)

axes[1].plot(list(k_range), davies_bouldin_scores_1_km, marker='o', color='orange')
axes[1].set_xlabel('Number of clusters (k)')
axes[1].set_ylabel('Davies-Bouldin Index')
axes[1].set_title('KMeans: Davies-Bouldin vs k')
axes[1].grid(True)

axes[2].plot(list(k_range), calinski_harabasz_scores_1_km, marker='o', color='green')
axes[2].set_xlabel('Number of clusters (k)')
axes[2].set_ylabel('Calinski-Harabasz Score')
axes[2].set_title('KMeans: Calinski-Harabasz vs k')
axes[2].grid(True)

plt.tight_layout()
plt.savefig('artifacts/figures/metrics_dataset1_kmeans.png', dpi=100, bbox_inches='tight')
plt.show()
print('Метрики KMeans сохранены')

## Датасет 2: Анализ и кластеризация

### Загрузка и EDA

In [ ]:
# Загрузка датасета 2
df2 = pd.read_csv('data/S07-hw-dataset-02.csv')

print('=== Датасет 2: Информация ===' )
print(f'Размер: {df2.shape}')
print(f'\nПервые строки:')
print(df2.head())
print(f'\nТип данных:')
print(df2.info())
print(f'\nСтатистика:')
print(df2.describe())
print(f'\nПропущенные значения:')
print(df2.isnull().sum())

### Предобработка датасета 2

In [ ]:
# Отделяем sample_id от признаков
sample_ids_2 = df2['sample_id'].copy()
X2_raw = df2.drop('sample_id', axis=1)

# Обработка пропущенных значений
imputer = SimpleImputer(strategy='mean')
X2_imputed = imputer.fit_transform(X2_raw)

# Стандартизация признаков
scaler = StandardScaler()
X2 = scaler.fit_transform(X2_imputed)

print('Датасет 2 предобработан')
print(f'Форма после предобработки: {X2.shape}')
print(f'Среднее: {X2.mean(axis=0)}')
print(f'Стандартное отклонение: {X2.std(axis=0)}')

### KMeans кластеризация для датасета 2

In [ ]:
# Тестирование KMeans с разными k
k_range = range(2, 21)
silhouette_scores_2_km = []
davies_bouldin_scores_2_km = []
calinski_harabasz_scores_2_km = []
kmeans_models_2 = {}

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_km = kmeans.fit_predict(X2)
    kmeans_models_2[k] = (kmeans, labels_km)
    
    sil_score = silhouette_score(X2, labels_km)
    db_score = davies_bouldin_score(X2, labels_km)
    ch_score = calinski_harabasz_score(X2, labels_km)
    
    silhouette_scores_2_km.append(sil_score)
    davies_bouldin_scores_2_km.append(db_score)
    calinski_harabasz_scores_2_km.append(ch_score)
    
    print(f'k={k}: Silhouette={sil_score:.4f}, Davies-Bouldin={db_score:.4f}, Calinski-Harabasz={ch_score:.4f}')

best_k_2_km = list(k_range)[np.argmax(silhouette_scores_2_km)]
print(f'\nЛучшее k для KMeans: {best_k_2_km}')
print(f'Лучший silhouette score: {max(silhouette_scores_2_km):.4f}')

### AgglomerativeClustering для датасета 2

In [ ]:
# Тестирование AgglomerativeClustering с разными k и linkage
k_range_agg = range(2, 21)
linkage_types = ['ward', 'complete']

silhouette_scores_2_agg = {}
davies_bouldin_scores_2_agg = {}
calinski_harabasz_scores_2_agg = {}
agg_models_2 = {}

for linkage in linkage_types:
    silhouette_scores_2_agg[linkage] = []
    davies_bouldin_scores_2_agg[linkage] = []
    calinski_harabasz_scores_2_agg[linkage] = []
    agg_models_2[linkage] = {}
    
    for k in k_range_agg:
        agg = AgglomerativeClustering(n_clusters=k, linkage=linkage)
        labels_agg = agg.fit_predict(X2)
        agg_models_2[linkage][k] = (agg, labels_agg)
        
        sil_score = silhouette_score(X2, labels_agg)
        db_score = davies_bouldin_score(X2, labels_agg)
        ch_score = calinski_harabasz_score(X2, labels_agg)
        
        silhouette_scores_2_agg[linkage].append(sil_score)
        davies_bouldin_scores_2_agg[linkage].append(db_score)
        calinski_harabasz_scores_2_agg[linkage].append(ch_score)
        
    print(f'Linkage {linkage}: лучший k = {np.argmax(silhouette_scores_2_agg[linkage]) + 2}, score = {max(silhouette_scores_2_agg[linkage]):.4f}')

### Визуализация датасета 2

In [ ]:
# PCA для визуализации
pca = PCA(n_components=2)
X2_pca = pca.fit_transform(X2)

# Лучший KMeans результат
labels_best_2_km = kmeans_models_2[best_k_2_km][1]

# PCA scatter plot для KMeans
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(X2_pca[:, 0], X2_pca[:, 1], c=labels_best_2_km, cmap='viridis', s=50, alpha=0.6)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
ax.set_title(f'Датасет 2: KMeans (k={best_k_2_km})')
plt.colorbar(scatter, ax=ax, label='Cluster')
plt.tight_layout()
plt.savefig('artifacts/figures/pca_dataset2_kmeans.png', dpi=100, bbox_inches='tight')
plt.show()
print('PCA plot KMeans сохранен')

In [ ]:
# Метрики vs K для Agglomerative
fig, axes = plt.subplots(1, 2, figsize=(14, 4))

for i, linkage in enumerate(linkage_types):
    axes[i].plot(list(k_range_agg), silhouette_scores_2_agg[linkage], marker='o', label=f'{linkage}')
    axes[i].set_xlabel('Number of clusters (k)')
    axes[i].set_ylabel('Silhouette Score')
    axes[i].set_title(f'Agglomerative ({linkage}): Silhouette vs k')
    axes[i].grid(True)

plt.tight_layout()
plt.savefig('artifacts/figures/metrics_dataset2_agg.png', dpi=100, bbox_inches='tight')
plt.show()
print('Метрики Agglomerative сохранены')

## Датасет 3: Анализ и кластеризация

### Загрузка и EDA

In [ ]:
# Загрузка датасета 3
df3 = pd.read_csv('data/S07-hw-dataset-03.csv')

print('=== Датасет 3: Информация ===' )
print(f'Размер: {df3.shape}')
print(f'\nПервые строки:')
print(df3.head())
print(f'\nТип данных:')
print(df3.info())
print(f'\nСтатистика:')
print(df3.describe())
print(f'\nПропущенные значения:')
print(df3.isnull().sum())

### Предобработка датасета 3

In [ ]:
# Отделяем sample_id от признаков
sample_ids_3 = df3['sample_id'].copy()
X3_raw = df3.drop('sample_id', axis=1)

# Обработка пропущенных значений
imputer = SimpleImputer(strategy='mean')
X3_imputed = imputer.fit_transform(X3_raw)

# Стандартизация признаков
scaler = StandardScaler()
X3 = scaler.fit_transform(X3_imputed)

print('Датасет 3 предобработан')
print(f'Форма после предобработки: {X3.shape}')
print(f'Среднее: {X3.mean(axis=0)}')
print(f'Стандартное отклонение: {X3.std(axis=0)}')

### KMeans кластеризация для датасета 3

In [ ]:
# Тестирование KMeans с разными k
k_range = range(2, 21)
silhouette_scores_3_km = []
davies_bouldin_scores_3_km = []
calinski_harabasz_scores_3_km = []
kmeans_models_3 = {}

for k in k_range:
    kmeans = KMeans(n_clusters=k, random_state=42, n_init=10)
    labels_km = kmeans.fit_predict(X3)
    kmeans_models_3[k] = (kmeans, labels_km)
    
    sil_score = silhouette_score(X3, labels_km)
    db_score = davies_bouldin_score(X3, labels_km)
    ch_score = calinski_harabasz_score(X3, labels_km)
    
    silhouette_scores_3_km.append(sil_score)
    davies_bouldin_scores_3_km.append(db_score)
    calinski_harabasz_scores_3_km.append(ch_score)
    
    print(f'k={k}: Silhouette={sil_score:.4f}, Davies-Bouldin={db_score:.4f}, Calinski-Harabasz={ch_score:.4f}')

best_k_3_km = list(k_range)[np.argmax(silhouette_scores_3_km)]
print(f'\nЛучшее k для KMeans: {best_k_3_km}')
print(f'Лучший silhouette score: {max(silhouette_scores_3_km):.4f}')

### DBSCAN кластеризация для датасета 3

In [ ]:
# Тестирование DBSCAN с разными eps
eps_range = np.linspace(0.2, 1.5, 15)
min_samples = 4

silhouette_scores_3_db = []
davies_bouldin_scores_3_db = []
calinski_harabasz_scores_3_db = []
noise_percentages_3_db = []
dbscan_models_3 = {}

for eps in eps_range:
    dbscan = DBSCAN(eps=eps, min_samples=min_samples)
    labels_db = dbscan.fit_predict(X3)
    dbscan_models_3[eps] = (dbscan, labels_db)
    
    n_clusters = len(set(labels_db)) - (1 if -1 in labels_db else 0)
    noise_points = np.sum(labels_db == -1)
    noise_percent = (noise_points / len(labels_db)) * 100
    noise_percentages_3_db.append(noise_percent)
    
    if n_clusters > 1 and len(set(labels_db)) > 1:
        sil_score = silhouette_score(X3, labels_db)
        db_score = davies_bouldin_score(X3, labels_db)
        ch_score = calinski_harabasz_score(X3, labels_db)
    else:
        sil_score = -1
        db_score = float('inf')
        ch_score = 0
    
    silhouette_scores_3_db.append(sil_score)
    davies_bouldin_scores_3_db.append(db_score)
    calinski_harabasz_scores_3_db.append(ch_score)
    
    print(f'eps={eps:.2f}: Clusters={n_clusters}, Noise={noise_percent:.2f}%, Silhouette={sil_score:.4f}')

best_eps_3_db = eps_range[np.argmax(silhouette_scores_3_db)]
print(f'\nЛучшее eps для DBSCAN: {best_eps_3_db:.2f}')
print(f'Лучший silhouette score: {max(silhouette_scores_3_db):.4f}')

### Визуализация датасета 3

In [ ]:
# PCA для визуализации
pca = PCA(n_components=2)
X3_pca = pca.fit_transform(X3)

# Лучший KMeans результат
labels_best_3_km = kmeans_models_3[best_k_3_km][1]

# PCA scatter plot для KMeans
fig, ax = plt.subplots(figsize=(10, 6))
scatter = ax.scatter(X3_pca[:, 0], X3_pca[:, 1], c=labels_best_3_km, cmap='viridis', s=50, alpha=0.6)
ax.set_xlabel(f'PC1 ({pca.explained_variance_ratio_[0]:.2%})')
ax.set_ylabel(f'PC2 ({pca.explained_variance_ratio_[1]:.2%})')
ax.set_title(f'Датасет 3: KMeans (k={best_k_3_km})')
plt.colorbar(scatter, ax=ax, label='Cluster')
plt.tight_layout()
plt.savefig('artifacts/figures/pca_dataset3_kmeans.png', dpi=100, bbox_inches='tight')
plt.show()
print('PCA plot KMeans сохранен')

In [ ]:
# Метрики vs eps для DBSCAN
fig, ax = plt.subplots(figsize=(10, 6))
ax.plot(eps_range, silhouette_scores_3_db, marker='o', color='purple')
ax.set_xlabel('eps parameter')
ax.set_ylabel('Silhouette Score')
ax.set_title('DBSCAN: Silhouette vs eps')
ax.grid(True)
plt.tight_layout()
plt.savefig('artifacts/figures/metrics_dataset3_dbscan.png', dpi=100, bbox_inches='tight')
plt.show()
print('Метрики DBSCAN сохранены')

## Проверка стабильности (Stability Check)

In [ ]:
# Проверка стабильности KMeans для датасета 1
# Запускаем KMeans 5 раз с разными random_state
random_states = [42, 123, 456, 789, 999]
stability_labels = []

print('=== Проверка стабильности KMeans (Датасет 1) ===' )
for rs in random_states:
    kmeans = KMeans(n_clusters=best_k_1_km, random_state=rs, n_init=10)
    labels = kmeans.fit_predict(X1)
    stability_labels.append(labels)
    print(f'Random state {rs}: KMeans выполнен')

# Сравнение результатов
ari_scores = []
print(f'\nARI scores между разными запусками:')
for i in range(len(stability_labels) - 1):
    ari = adjusted_rand_score(stability_labels[i], stability_labels[i+1])
    ari_scores.append(ari)
    print(f'Run {i+1} vs Run {i+2}: ARI = {ari:.4f}')

print(f'\nСредний ARI score: {np.mean(ari_scores):.4f}')
print(f'Минимальный ARI score: {np.min(ari_scores):.4f}')
print(f'Максимальный ARI score: {np.max(ari_scores):.4f}')

## Сохранение меток кластеров

In [ ]:
# Сохранение меток для датасета 1
labels_df1 = pd.DataFrame({
    'sample_id': sample_ids_1,
    'cluster_label': labels_best_1_km
})
labels_df1.to_csv('artifacts/labels/labels_hw07_ds1.csv', index=False)
print('Датасет 1 метки сохранены')

# Сохранение меток для датасета 2
labels_df2 = pd.DataFrame({
    'sample_id': sample_ids_2,
    'cluster_label': labels_best_2_km
})
labels_df2.to_csv('artifacts/labels/labels_hw07_ds2.csv', index=False)
print('Датасет 2 метки сохранены')

# Сохранение меток для датасета 3
labels_df3 = pd.DataFrame({
    'sample_id': sample_ids_3,
    'cluster_label': labels_best_3_km
})
labels_df3.to_csv('artifacts/labels/labels_hw07_ds3.csv', index=False)
print('Датасет 3 метки сохранены')

## Сохранение метрик в JSON

In [ ]:
# Создание объекта со всеми метриками
metrics_summary = {
    'dataset_1': {
        'KMeans': {
            'best_k': int(best_k_1_km),
            'metrics': {
                'silhouette_score': float(max(silhouette_scores_1_km)),
                'davies_bouldin_score': float(min(davies_bouldin_scores_1_km)),
                'calinski_harabasz_score': float(max(calinski_harabasz_scores_1_km))
            }
        },
        'DBSCAN': {
            'best_eps': float(best_eps_1_db),
            'min_samples': 5,
            'metrics': {
                'silhouette_score': float(max(silhouette_scores_1_db)),
                'davies_bouldin_score': float(min([x for x in davies_bouldin_scores_1_db if x != float('inf')])),
                'noise_percentage': float(max([noise_percentages_1_db[i] for i in range(len(noise_percentages_1_db)) if silhouette_scores_1_db[i] == max(silhouette_scores_1_db)]))
            }
        }
    },
    'dataset_2': {
        'KMeans': {
            'best_k': int(best_k_2_km),
            'metrics': {
                'silhouette_score': float(max(silhouette_scores_2_km)),
                'davies_bouldin_score': float(min(davies_bouldin_scores_2_km)),
                'calinski_harabasz_score': float(max(calinski_harabasz_scores_2_km))
            }
        },
        'AgglomerativeClustering': {
            'best_linkage': 'ward' if max(silhouette_scores_2_agg['ward']) > max(silhouette_scores_2_agg['complete']) else 'complete',
            'best_k': int(np.argmax(silhouette_scores_2_agg['ward' if max(silhouette_scores_2_agg['ward']) > max(silhouette_scores_2_agg['complete']) else 'complete']) + 2),
            'metrics': {
                'silhouette_score': float(max(max(silhouette_scores_2_agg['ward']), max(silhouette_scores_2_agg['complete']))),
                'davies_bouldin_score': float(min(min(davies_bouldin_scores_2_agg['ward']), min(davies_bouldin_scores_2_agg['complete']))),
                'calinski_harabasz_score': float(max(max(calinski_harabasz_scores_2_agg['ward']), max(calinski_harabasz_scores_2_agg['complete'])))
            }
        }
    },
    'dataset_3': {
        'KMeans': {
            'best_k': int(best_k_3_km),
            'metrics': {
                'silhouette_score': float(max(silhouette_scores_3_km)),
                'davies_bouldin_score': float(min(davies_bouldin_scores_3_km)),
                'calinski_harabasz_score': float(max(calinski_harabasz_scores_3_km))
            }
        },
        'DBSCAN': {
            'best_eps': float(best_eps_3_db),
            'min_samples': 4,
            'metrics': {
                'silhouette_score': float(max(silhouette_scores_3_db)),
                'davies_bouldin_score': float(min([x for x in davies_bouldin_scores_3_db if x != float('inf')])),
                'noise_percentage': float(max([noise_percentages_3_db[i] for i in range(len(noise_percentages_3_db)) if silhouette_scores_3_db[i] == max(silhouette_scores_3_db)]))
            }
        }
    }
}

# Сохранение в JSON файл
with open('artifacts/metrics_summary.json', 'w') as f:
    json.dump(metrics_summary, f, indent=2)

print('Метрики сохранены в JSON')

In [ ]:
# Создание объекта с лучшими конфигурациями
best_configs = {
    'dataset_1': {
        'algorithm': 'KMeans',
        'parameters': {
            'n_clusters': int(best_k_1_km),
            'random_state': 42,
            'n_init': 10
        },
        'performance': {
            'silhouette_score': float(max(silhouette_scores_1_km))
        }
    },
    'dataset_2': {
        'algorithm': 'AgglomerativeClustering',
        'parameters': {
            'n_clusters': int(np.argmax(silhouette_scores_2_agg['ward']) + 2),
            'linkage': 'ward'
        },
        'performance': {
            'silhouette_score': float(max(silhouette_scores_2_agg['ward']))
        }
    },
    'dataset_3': {
        'algorithm': 'DBSCAN',
        'parameters': {
            'eps': float(best_eps_3_db),
            'min_samples': 4
        },
        'performance': {
            'silhouette_score': float(max(silhouette_scores_3_db))
        }
    }
}

# Сохранение в JSON файл
with open('artifacts/best_configs.json', 'w') as f:
    json.dump(best_configs, f, indent=2)

print('Конфигурации сохранены в JSON')

## Итоговый отчет

In [ ]:
print('=== ИТОГОВЫЙ ОТЧЕТ HW07 ===')
print()
print('ДАТАСЕТ 1: Числовые признаки с разными масштабами')
print(f'  Выбранный метод: KMeans (k={best_k_1_km})')
print(f'  Лучший silhouette score: {max(silhouette_scores_1_km):.4f}')
print(f'  Вызовы: различные масштабы признаков, необходима стандартизация')
print()
print('ДАТАСЕТ 2: Нелинейная структура с выбросами')
best_link = 'ward' if max(silhouette_scores_2_agg['ward']) > max(silhouette_scores_2_agg['complete']) else 'complete'
print(f'  Выбранный метод: AgglomerativeClustering ({best_link} linkage)')
print(f'  Лучший k: {np.argmax(silhouette_scores_2_agg[best_link]) + 2}')
print(f'  Лучший silhouette score: {max(silhouette_scores_2_agg[best_link]):.4f}')
print(f'  Вызовы: иерархическая кластеризация лучше справляется с нелинейностью')
print()
print('ДАТАСЕТ 3: Кластеры с разной плотностью')
print(f'  Выбранный метод: DBSCAN (eps={best_eps_3_db:.2f})')
print(f'  Лучший silhouette score: {max(silhouette_scores_3_db):.4f}')
print(f'  Вызовы: DBSCAN способен обнаруживать кластеры произвольной формы')
print()
print('ПРОВЕРКА СТАБИЛЬНОСТИ:')
print(f'  Датасет 1 (KMeans): средний ARI = {np.mean(ari_scores):.4f}')
print(f'  KMeans показывает хорошую стабильность')
print()
print('=== ФАЙЛЫ СОХРАНЕНЫ ===')
print('  - artifacts/figures/: 6 PNG изображений')
print('  - artifacts/labels/: 3 CSV файла с метками')
print('  - artifacts/metrics_summary.json')
print('  - artifacts/best_configs.json')